In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv("spam_ham_3000.csv")

In [23]:
df

,id,label,timestamp,subject,body
0,S1398,1,2025-12-20 19:45:00,환급금 10000원 확인하세요 (마감 18시),결제 오류가 감지되었습니다. 계정 확인을 위해 https://me2.do/93OE7...
1,S0309,1,2026-02-04 13:58:00,신용등급 영향 없이 50만원 대출 가능,한정 이벤트: 지금 가입하면 80% 할인 + 배송비 무료 지급! https://ti...
2,S0441,1,2025-12-29 21:11:00,환급금 70000원 확인하세요 (마감 12시),한정 이벤트: 지금 가입하면 30% 할인 + 쿠폰팩 지급! https://tinyu...
3,S0760,1,2025-12-30 20:03:00,브랜드A 단독특가! 오늘만 50% OFF,오늘만 특가! 브랜드A 인기상품 50% OFF. 재고 소진 전 주문: https:/...
4,S1117,1,2026-02-22 04:02:00,신용등급 영향 없이 300만원 대출 가능,BTC 급등 예상 리포트 무료 배포. 단 20시 이전 신청자만. https://na...
...,...,...,...,...,...
2995,H1012,0,2026-01-11 05:21:00,계약서 초안 검토 부탁,"준호님,\n\n주가예측 PR 올렸습니다. 시간 되실 때 코드리뷰 부탁드립니다.\n주..."
2996,H0966,0,2026-01-28 22:05:00,[공지] 오프사이트 일정 안내,"준호님,\n\n스팸필터 진행 상황 업데이트 드립니다.\n현재 테스트 완료이며, 다음..."
2997,S0163,1,2025-12-22 00:10:00,신용등급 영향 없이 300만원 대출 가능,신용조회 없이 300만원 가능. 상담 신청: https://bit.ly/NBmOXr...
2998,S0530,1,2026-02-17 17:13:00,지금 가입하면 40% 할인 + 배송비 무료,"KB 사전승인 완료. 최대 100만원, 금리 3.5%~. 서류 없이 진행: http..."


In [9]:
df_subject = df.subject.tolist()

In [6]:
# replace는 정의되지 않은 값은 원래대로 유지합니다.
df['label'] = df['label'].replace({'spam': 1, 'ham': 0})

C:\Users\playdata2\AppData\Local\Temp\ipykernel_19780\4253789291.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'spam': 1, 'ham': 0})


In [7]:
labels = df.label.tolist()

In [10]:
emails = df_subject

labels = df.label.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    emails, labels, test_size=0.25, random_state=42
)

In [11]:
# 한국어에서 강력한 기본 베이스라인 (형태소 없이도 성능 좋음)
vectorizer = CountVectorizer(analyzer="char", ngram_range=(2,4))

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("학습 데이터 shape:", X_train_vec.shape)

학습 데이터 shape: (2250, 2375)


In [12]:
model = MultinomialNB(alpha=1.0)  # 라플라스 스무딩


model.fit(X_train_vec, y_train)


preds = model.predict(X_test_vec)


print("정확도:", accuracy_score(y_test, preds))
print("혼동행렬:")
print(confusion_matrix(y_test, preds))


정확도: 1.0
혼동행렬:
[[370   0]
 [  0 380]]


In [13]:
proba = model.predict_proba(X_test_vec)

for mail, p in zip(X_test, proba):
    print(f"메일: {mail}")
    print(f"  스팸 확률: {p[1]:.3f}")


메일: 브랜드D 단독특가! 오늘만 60% OFF
  스팸 확률: 1.000
메일: 무료체험 30일! 지금 클릭하면 혜택
  스팸 확률: 1.000
메일: 즉시입금 30000원! 이벤트 참여하세요
  스팸 확률: 1.000
메일: 무료체험 14일! 지금 클릭하면 혜택
  스팸 확률: 1.000
메일: 딥러닝 과제 제출 안내
  스팸 확률: 0.000
메일: 무료체험 14일! 지금 클릭하면 혜택
  스팸 확률: 1.000
메일: 당신만을 위한 SOL 투자 비밀 공개
  스팸 확률: 1.000
메일: [알림] 결제 실패 — 재시도 필요
  스팸 확률: 1.000
메일: 즉시입금 10000원! 이벤트 참여하세요
  스팸 확률: 1.000
메일: [공지] 런칭 미팅 일정 안내
  스팸 확률: 0.000
메일: 주간 보고 (45주차)
  스팸 확률: 0.000
메일: [알림] 결제 실패 — 계정잠금 해제 필요
  스팸 확률: 1.000
메일: 데이터사이언스 과제 제출 안내
  스팸 확률: 0.000
메일: 신용등급 영향 없이 300만원 대출 가능
  스팸 확률: 1.000
메일: [공지] 정기 회의 일정 안내
  스팸 확률: 0.000
메일: 브랜드B 단독특가! 오늘만 50% OFF
  스팸 확률: 1.000
메일: 무료체험 3일! 지금 클릭하면 혜택
  스팸 확률: 1.000
메일: [공지] 교육 세션 일정 안내
  스팸 확률: 0.000
메일: 당신만을 위한 BTC 투자 비밀 공개
  스팸 확률: 1.000
메일: [알림] 결제 실패 — 재시도 필요
  스팸 확률: 1.000
메일: 로그분석 코드리뷰 요청드립니다
  스팸 확률: 0.000
메일: 데이터팀 회의록 공유 (2026-02-23)
  스팸 확률: 0.000
메일: 지훈님, 일정 관련 확인 부탁드립니다
  스팸 확률: 0.000
메일: 추천시스템 코드리뷰 요청드립니다
  스팸 확률: 0.000
메일: 영업팀 회의록 공유 (2026-02-26)
  스팸 확률: 0.000
메일: 계약서 초안 검토 부탁
  스팸 

In [15]:
def check_spam(new_texts):
    # 1. 벡터화
    vec = vectorizer.transform(new_texts)
    # 2. 예측
    pred = model.predict(vec)
    # 3. 확률
    prob = model.predict_proba(vec)
    
    for text, p, label in zip(new_texts, prob, pred):
        kind = "스팸" if label == 1 else "정상"
        print(f"[{kind}] (확률: {p[1]:.4f}) -> {text}")

# 테스트!
test_samples = [
    "부장님, 오늘 점심은 김치찌개 어떠신가요?", 
    "최저 금리 보장! 지금 바로 한도 조회하세요. http://loan.com",
    "이벤트 당첨을 축하드립니다. 상품권 수령하기"
]
check_spam(test_samples)

[정상] (확률: 0.0000) -> 부장님, 오늘 점심은 김치찌개 어떠신가요?
[스팸] (확률: 1.0000) -> 최저 금리 보장! 지금 바로 한도 조회하세요. http://loan.com
[스팸] (확률: 1.0000) -> 이벤트 당첨을 축하드립니다. 상품권 수령하기


In [16]:
import pandas as pd

# 1. 벡터라이저에서 단어(글자조합) 목록 가져오기
feature_names = vectorizer.get_feature_names_out()

# 2. 모델이 학습한 각 단어별 스팸 확률(log probability) 가져오기
# model.feature_log_prob_[1]은 '스팸(1)' 클래스에 대한 가중치입니다.
spam_weights = model.feature_log_prob_[1]

# 3. 보기 좋게 데이터프레임으로 만들기
features_df = pd.DataFrame({
    '특징(글자조합)': feature_names,
    '가중치': spam_weights
})

# 4. 가중치가 높은 순(스팸일 확률이 높은 순)으로 정렬해서 출력
print("### 모델이 생각하는 '스팸' 핵심 키워드 TOP 20 ###")
print(features_df.sort_values(by='가중치', ascending=False).head(20))

### 모델이 생각하는 '스팸' 핵심 키워드 TOP 20 ###
     특징(글자조합)       가중치
642        00 -4.292482
643       000 -4.914349
1288       금  -4.982512
422        !  -5.066886
1873       원  -5.156569
63          1 -5.476876
1532       만원 -5.494029
1533      만원  -5.494029
644      0000 -5.529244
682      0만원  -5.580726
680        0만 -5.580726
681       0만원 -5.580726
2046       즉시 -5.651068
341         지 -5.655124
648       00원 -5.688185
2272       하세 -5.688185
1692       세요 -5.688185
2273      하세요 -5.688185
685        0원 -5.688185
645      000원 -5.688185


In [21]:
weaktest = [
    "광고] 이번주 로또 예상 번호 무료 전송. 지금 바로 답장 주세요.", # 링크 없음
    "내일 오전 10시 회의 장소는 https://zoom.us/j/12345 입니다.", # 링크 있는 정상 메일
    "안녕하세요, 확인 부탁드립니다."
]
check_spam(weaktest)

[스팸] (확률: 1.0000) -> 광고] 이번주 로또 예상 번호 무료 전송. 지금 바로 답장 주세요.
[정상] (확률: 0.0000) -> 내일 오전 10시 회의 장소는 https://zoom.us/j/12345 입니다.
[정상] (확률: 0.0000) -> 안녕하세요, 확인 부탁드립니다.


In [22]:
final_test = [
    "안녕하세요, 대출 금리 안내드립니다.",  # [정상 단어] + [스팸 단어] 섞임
    "[광고] 부장님, 오늘 점심은 김치찌개 어떠신가요?", # [스팸 태그] + [정상 문장] 섞임
    "000원 확인 부탁드립니다." # [스팸 단어] + [정상 단어] 섞임
]
check_spam(final_test)

[정상] (확률: 0.0011) -> 안녕하세요, 대출 금리 안내드립니다.
[정상] (확률: 0.0000) -> [광고] 부장님, 오늘 점심은 김치찌개 어떠신가요?
[정상] (확률: 0.0000) -> 000원 확인 부탁드립니다.
